<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select COUNT(id) 
                from vacancies'''
count_vacancies = pd.read_sql_query(query_3_1, connection)

In [5]:
print('Количество вакансий в базе:', count_vacancies['count'].iloc[0])

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_3_2 = f'''select COUNT(id) 
                from employers'''
employers = pd.read_sql_query(query_3_2, connection)

In [7]:
print('Количество работодателей в базе:', employers['count'].iloc[0])

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_3_3 = f'''select COUNT(id) 
                from areas'''
areas = pd.read_sql_query(query_3_3, connection)

In [9]:
print('Количество регионов в базе:', areas['count'].iloc[0])

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_3_4 = f'''select COUNT(id) 
                from industries'''
industries = pd.read_sql_query(query_3_4, connection)

In [11]:
print('Количество индустрий в базе:', industries['count'].iloc[0])

Количество индустрий в базе: 294


***

# Выводы
1. В нашем распоряжении достаточно обширная база, состоящая из 49197 вакансий
2. В базе представлены вакансии от 23501 работодателя
3. Имеет широкую географическую (1362 региона) и индустриальную(294 индустрии) представленность 
4. Можно говорить о репрезентативной выборке и экстраполировать выводы на рынок в целом

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''SELECT a.name area,
                       COUNT(v.id) AS cnt_vacancies
                FROM vacancies AS v INNER JOIN areas AS a ON v.area_id = a.id
                GROUP BY 1
                ORDER BY 2 DESC'''
vac_by_areas = pd.read_sql_query(query_4_1, connection)

In [13]:
print('Пятерка лидеров по количеству вакансий:', '', vac_by_areas.head(5), sep='\n')

Пятерка лидеров по количеству вакансий:

              area  cnt_vacancies
0           Москва           5333
1  Санкт-Петербург           2851
2            Минск           2112
3      Новосибирск           2006
4           Алматы           1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''SELECT 
                       COUNT(id) AS cnt
                FROM vacancies
                WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
                '''

has_salary = pd.read_sql_query(query_4_2, connection)

In [15]:
print('Заполнено хотя бы одно из двух полей с зарплатой у {} вакансий. Что составляет {}% от общего числа вакансий.'.format(has_salary['cnt'].iloc[0],round(has_salary['cnt'].iloc[0]/count_vacancies['count'].iloc[0]*100,0)))

Заполнено хотя бы одно из двух полей с зарплатой у 24073 вакансий. Что составляет 49.0% от общего числа вакансий.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_4_3 = f'''SELECT 
                       ROUND(AVG(salary_from),0) AS avg_salary_from,
                       ROUND(AVG(salary_to),0) AS avg_salary_to
                FROM vacancies
                WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
                '''

avg_salary = pd.read_sql_query(query_4_3, connection)

In [17]:
print('Cредние значения для нижней границы зарплатной вилки {}, для верхней границы {}.'.format(avg_salary['avg_salary_from'].iloc[0], avg_salary['avg_salary_to'].iloc[0]))

Cредние значения для нижней границы зарплатной вилки 71065.0, для верхней границы 110537.0.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_4_4 = f'''SELECT schedule,
                       employment,
                       COUNT(id) AS cnt    
                FROM vacancies
                GROUP BY 1,2
                ORDER BY 3 DESC
                '''

vac_by_sch = pd.read_sql_query(query_4_4, connection)

In [19]:
vac_by_sch

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''SELECT experience   
                FROM vacancies
                GROUP BY 1
                ORDER BY COUNT(id) ASC
                '''

vac_by_experience = pd.read_sql_query(query_4_5, connection)

In [21]:
print('Требуемый опыт работы в порядке возрастания количества вакансий:', '', vac_by_experience, sep='\n')

Требуемый опыт работы в порядке возрастания количества вакансий:

           experience
0         Более 6 лет
1           Нет опыта
2       От 3 до 6 лет
3  От 1 года до 3 лет


***

# Выводы 
1. Больше всего вакансий в крупных столичных городах, что обусловлено концентрацией работодателей в столицах и количеством населения в крупных городах
2. В Москве вакансий практически в 2 раза больше по сравнению с другими городами, попавшими в топ 5
3. Лишь у половины вакансий(24073) заполнено хотя бы одно поле с зарплатой
4. Cредние значения зарплат лежит в диапазоне от 71065 до 110537
5. В основном работодатель ищет сотрудников на полный день и полную занятость
6. Самое редкое сочетание занятости и расписания сменный график - проектная работа
7. Самый востребованный опыт работы 1-3 лет, люди с опытом более 6 лет востребованы меньше всего 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_5_1 = f'''SELECT e.name                      
                FROM vacancies AS v INNER JOIN employers AS e ON v.employer_id = e.id
                GROUP BY 1
                ORDER BY COUNT(v.id) DESC
                LIMIT 5'''
vac_by_employers = pd.read_sql_query(query_5_1, connection)

In [23]:
print('На первом месте по количеству вакансий находится {}. На пятом месте по количеству вакансий находится {}.'.format(vac_by_employers['name'].iloc[0],vac_by_employers['name'].iloc[4]))

На первом месте по количеству вакансий находится Яндекс. На пятом месте по количеству вакансий находится Газпром нефть.


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_5_2 = f'''SELECT a.name,
                COUNT(e.id) AS cnt_employes,
                COUNT(V.id) AS cnt_vacancies
                FROM areas AS a LEFT JOIN  vacancies AS v ON v.area_id = a.id
                LEFT JOIN employers AS e ON e.area = a.id
                GROUP BY 1
                ORDER BY 3 NULLS FIRST, 2 DESC
                LIMIT 1'''
areas_cnt_emp_vac = pd.read_sql_query(query_5_2, connection)

In [25]:
areas_cnt_emp_vac

,name,cnt_employes,cnt_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_5_3 = f'''SELECT e.name,
                COUNT(DISTINCT a.id)
                FROM vacancies AS v INNER JOIN employers AS e ON v.employer_id = e.id
                INNER JOIN areas AS a ON v.area_id = a.id
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 5'''
employers_cnt_areas = pd.read_sql_query(query_5_3, connection)


In [27]:
employers_cnt_areas

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query_5_4 = f'''SELECT COUNT(e.id)
                FROM employers AS e
                WHERE e.id NOT IN (SELECT employer_id FROM EMPLOYERS_INDUSTRIES)'''
employers_wo_industries = pd.read_sql_query(query_5_4, connection)


In [29]:
print('Количество работодателей, у которых не указана сфера деятельности:', employers_wo_industries['count'].iloc[0])

Количество работодателей, у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query_5_5 = f'''SELECT e.name
                FROM employers AS e INNER JOIN employers_industries AS ei ON e.id = ei.employer_id
                GROUP BY 1
                HAVING COUNT(ei.industry_id) = 4
                ORDER BY 1 ASC
                OFFSET 2 LIMIT 1
                '''
employers_industries = pd.read_sql_query(query_5_5, connection)

In [31]:
print('Компания, находящаяся на третьем месте в алфавитном списке компаний, у которых указано четыре сферы деятельности:', employers_industries['name'].iloc[0])

Компания, находящаяся на третьем месте в алфавитном списке компаний, у которых указано четыре сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
query_5_6 = f'''SELECT COUNT(DISTINCT e.id)
                FROM employers AS e INNER JOIN employers_industries AS ei ON e.id = ei.employer_id
                INNER JOIN industries AS i ON i.id = ei.industry_id
                WHERE i.name = 'Разработка программного обеспечения'
                ORDER BY 1 ASC
                '''
employers_industries_po = pd.read_sql_query(query_5_6, connection)

In [33]:
print('У {} работодателей в качестве сферы деятельности указана Разработка программного обеспечения.'.format(employers_industries_po['count'].iloc[0]))

У 3553 работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
df = pd.read_html(url)[1]
t = tuple(df['Город'].to_list())

In [35]:
query_5_7 = f'''
                (SELECT a.name city,
                COUNT(v.id) cnt_vacancies
                FROM vacancies AS v INNER JOIN employers AS e ON v.employer_id = e.id
                INNER JOIN areas AS a ON v.area_id = a.id
                WHERE e.name = 'Яндекс'
                AND a.name in {t}
                GROUP BY 1
                ORDER BY 2 DESC)
                
                UNION ALL
                
                (SELECT 'Total',
                COUNT(v.id)
                FROM vacancies AS v INNER JOIN employers AS e ON v.employer_id = e.id
                INNER JOIN areas AS a ON v.area_id = a.id
                WHERE e.name = 'Яндекс'
                AND a.name in {t}
                GROUP BY 1)
                ORDER BY 2 ASC'''
employers_cnt_areas = pd.read_sql_query(query_5_7, connection)

In [36]:
employers_cnt_areas

,city,cnt_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Казань,25
5,Ростов-на-Дону,25
6,Пермь,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

# Выводы
1. Лидером по количество вакансий является Яндекс, замыкает пятерку лидеров Газпром нефть. Очевидно, что лидеры рынка постоянно нуждаются в кадрах
2. В базе есть работодатели, которые не имеют открытых вакансий на текущий момент, большинство таких работодателей находятся в России 
3. Яндекс имеет самую обширную географию поиска сотрудников
4. 8419 работодателей не определились со своей сферой деятельности
5. У 15% (3553) работодателей в качестве сферы деятельности указана Разработка программного обеспечения.
6. Компания Яндекс ищет 485 сотрудников, самый большой набор идет в Москве и Санкт-Петербурге

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''select COUNT(id) 
                from vacancies
                WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%'
                '''
data_vac = pd.read_sql_query(query_6_1, connection)

In [38]:
print('{} вакансий имеет отношение к данным.'.format(data_vac['count'].iloc[0]))

1771 вакансий имеет отношение к данным.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query_6_2 = f'''select COUNT(id) 
                from vacancies
                WHERE (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      AND (LOWER(name) LIKE '%junior%'
                      OR experience = 'Нет опыта'
                      OR employment = 'Стажировка'
                      )
                      
                '''
query_6_2_1 = f'''select COUNT(id) 
                from vacancies
                WHERE (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      
                      
                '''
ds_vac_jun = pd.read_sql_query(query_6_2, connection)
ds_vac = pd.read_sql_query(query_6_2_1, connection)

In [40]:
# результат запроса
print('{} из {} вакансий подходят для начинающего дата-сайентиста.'.format(ds_vac_jun['count'].iloc[0],ds_vac['count'].iloc[0] ))
print('Это составляет {}% от общего числа вакансий для дата-сайентиста.'.format(round(ds_vac_jun['count'].iloc[0]/ds_vac['count'].iloc[0]*100,0)))

51 из 480 вакансий подходят для начинающего дата-сайентиста.
Это составляет 11.0% от общего числа вакансий для дата-сайентиста.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''select COUNT(id) 
                from vacancies
                WHERE (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      AND (LOWER(key_skills) LIKE '%sql%'
                      OR LOWER(key_skills) LIKE '%postgres%')
                      
                '''
ds_sql_pg_vac = pd.read_sql_query(query_6_3, connection)


In [42]:
print('{} вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres.'.format(ds_sql_pg_vac['count'].iloc[0]))
print('Что составляет {}% от общего числа DS вакансий'.format(round(ds_sql_pg_vac['count'].iloc[0]/ds_vac['count'].iloc[0]*100,0)))

201 вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres.
Что составляет 42.0% от общего числа DS вакансий


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
query_6_4 = f'''select COUNT(id) 
                from vacancies
                WHERE (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      AND (LOWER(key_skills) LIKE '%python%'
                      )
                      
                '''
ds_py_vac = pd.read_sql_query(query_6_4, connection)

In [44]:
print('{} вакансий, в которых в качестве ключевого навыка указан Python.'.format(ds_py_vac['count'].iloc[0]))
print('Что составляет {}% от общего числа DS вакансий'.format(round(ds_py_vac['count'].iloc[0]/ds_vac['count'].iloc[0]*100,0)))

351 вакансий, в которых в качестве ключевого навыка указан Python.
Что составляет 73.0% от общего числа DS вакансий


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query_6_5 = f'''SELECT 
                ROUND(AVG(CHAR_LENGTH(key_skills) - CHAR_LENGTH(REPLACE(key_skills, CHR(9),''))+1),2) count
                from vacancies
                WHERE (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      AND (key_skills IS NOT NULL OR LOWER(key_skills) NOT LIKE '%none%' )
                      
                '''
ds_key_skills_cnt = pd.read_sql_query(query_6_5, connection)

query_6_5_1 = f'''SELECT 
                ROUND(AVG(CHAR_LENGTH(key_skills) - CHAR_LENGTH(REPLACE(key_skills, CHR(9),''))+1),2) count
                from vacancies
                WHERE  (key_skills IS NOT NULL OR LOWER(key_skills) NOT LIKE '%none%' )
                      
               '''
key_skills_cnt = pd.read_sql_query(query_6_5_1, connection)


In [46]:
print('В среднем в вакансиях для DS указывают {} ключевых навыков.'.format(ds_key_skills_cnt['count'].iloc[0]))
print('В среднем в вакансиях указывают {} ключевых навыков.'.format(key_skills_cnt['count'].iloc[0]))

В среднем в вакансиях для DS указывают 6.41 ключевых навыков.
В среднем в вакансиях указывают 6.37 ключевых навыков.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query_6_6 = f'''SELECT experience,
                AVG(CASE WHEN salary_from IS NULL THEN salary_to
                     WHEN salary_to IS NULL THEN salary_from
                     ELSE (salary_from+salary_to)/2
                END)
                FROM vacancies
                WHERE 
                      (LOWER(name) LIKE '%data scientist%' 
                      OR LOWER(name) LIKE '%data science%'
                      OR LOWER(name) LIKE '%исследователь данных%'
                      OR LOWER(name) LIKE '%machine learning%'
                      OR LOWER(name) LIKE '%машинн%обучен%'
                      OR (name LIKE '%ML%'
                      AND name NOT LIKE '%HTML%'))
                      AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                GROUP BY 1
                '''

avg_salary = pd.read_sql_query(query_6_6, connection)

In [48]:
avg_salary

,experience,avg
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


In [49]:
#Посмотрим, в каком регионе зарплата начинающего DS выше
query_6_6_1 = f'''SELECT a.name,
                AVG(CASE WHEN salary_from IS NULL THEN salary_to
                     WHEN salary_to IS NULL THEN salary_from
                     ELSE (salary_from+salary_to)/2
                END)
                FROM vacancies AS v INNER JOIN areas AS a ON v.area_id = a.id
                WHERE 
                      (LOWER(v.name) LIKE '%data scientist%' 
                      OR LOWER(v.name) LIKE '%data science%'
                      OR LOWER(v.name) LIKE '%исследователь данных%'
                      OR LOWER(v.name) LIKE '%machine learning%'
                      OR LOWER(v.name) LIKE '%машинн%обучен%'
                      OR (v.name LIKE '%ML%'
                      AND v.name NOT LIKE '%HTML%'))
                      AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                      AND (LOWER(v.name) LIKE '%junior%'
                      OR experience = 'Нет опыта'
                      OR employment = 'Стажировка'
                      )
                GROUP BY 1
                ORDER BY 2 DESC
                '''

avg_salary_by_area = pd.read_sql_query(query_6_6_1, connection)

In [50]:
avg_salary_by_area

,name,avg
0,Санкт-Петербург,87666.666667
1,Новосибирск,80000.000000
2,Москва,72500.000000
3,Владивосток,70000.000000
4,Пермь,60000.000000
5,Ростов-на-Дону,50000.000000
6,Екатеринбург,37000.000000


In [51]:
#Посмотрим, в каком регионе самые дорогие DS
query_6_6_3 = f'''SELECT a.name,
                AVG(CASE WHEN salary_from IS NULL THEN salary_to
                     WHEN salary_to IS NULL THEN salary_from
                     ELSE (salary_from+salary_to)/2
                END)
                FROM vacancies AS v INNER JOIN areas AS a ON v.area_id = a.id
                WHERE 
                      (LOWER(v.name) LIKE '%data scientist%' 
                      OR LOWER(v.name) LIKE '%data science%'
                      OR LOWER(v.name) LIKE '%исследователь данных%'
                      OR LOWER(v.name) LIKE '%machine learning%'
                      OR LOWER(v.name) LIKE '%машинн%обучен%'
                      OR (v.name LIKE '%ML%'
                      AND v.name NOT LIKE '%HTML%'))
                      AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                      
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 10
                '''

avg_salary_by_area = pd.read_sql_query(query_6_6_3, connection)
avg_salary_by_area

,name,avg
0,Кипр,300000.000000
1,Армения,268862.666667
2,Черногория,233794.000000
3,Турция,233794.000000
4,Сербия,233794.000000
5,Москва,207629.629630
6,Белгород,200000.000000
7,Санкт-Петербург,173335.142857
8,Новосибирск,166333.333333
9,Рязань,136666.666667


In [52]:
#Также интересно посмотреть, в каких индустриях нанимают DS
query_6_6_2 = f'''SELECT i.name,
                COUNT(v.id)
                from vacancies AS v INNER JOIN employers AS e ON v.employer_id = e.id
                INNER JOIN employers_industries AS ei ON e.id = ei.employer_id
                INNER JOIN industries AS i ON i.id = ei.industry_id
                WHERE (LOWER(v.name) LIKE '%data scientist%' 
                      OR LOWER(v.name) LIKE '%data science%'
                      OR LOWER(v.name) LIKE '%исследователь данных%'
                      OR LOWER(v.name) LIKE '%machine learning%'
                      OR LOWER(v.name) LIKE '%машинн%обучен%'
                      OR (v.name LIKE '%ML%'
                      AND v.name NOT LIKE '%HTML%'))
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 10
  
                '''
ds_ind = pd.read_sql_query(query_6_6_2, connection)
ds_ind

,name,count
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,92
3,"Интернет-компания (поисковики, платежные систе...",79
4,Мобильная связь,23
5,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22
6,"Управляющая, инвестиционная компания (управлен...",21
7,Консалтинговые услуги,16
8,Фиксированная связь,13
9,"Научно-исследовательская, научная, академическ...",9


***

# Выводы
1. 1771 вакансий имеет отношение к данным
2. 480 вакансий в базе подходит для дата-сайентиста, из них лишь 11% подходит для начинающего специалиста
3. В среднем в вакансиях для DS указывается 6,41 ключевой навыков, среднем список ключевых навыков состоит их 6,37, т.е от DS ожидают немного большего, чем в среднем по рынку труда
4. Python является востребованным навыком и упоминается в 73% вакансий, SQL также необходим дата-сайентисту в 42% вакансий
5. Зарплата дата-сайентиста растет вместе с опытом, стартовая средняя зарплата составлет 74643 руб. 
6. Самые высокие зарплаты в DS подучают новички из Санкт-Петербурга и Новосибирска
7. DS ищут в основном компании связанные с IT (Разработка ПО, системная интеграция), однако в банковской сфере также идет активный поиск

# Общий вывод по проекту

1. 1% вакансий в базе данных подходит для DS
2. Дата-сайентист должен расширять свой стек, так как список ключевых навыков, указанный в вакансиях, состоит в среднем из 6,41 пункта (в дальнейшем исследовании нужно вывести список и количество упоминаний ключевых навыков для Дата-сайентиста)
3. Python является ключевой компетенцией Дата-сайентиста и требуется в 73% случаев
4. SQL также востребован
4. Зарплата дата-сайентиста растет вместе с опытом, обращает внимание, что DS специалистов с опытом больше 6 лет не ищут, возможно это связано с молодостью профессии, однако имеет смысл проверить, куда вырастают опытные специалисты
5. Самые высокие зарплаты предлагают работодатели за пределами России